In [46]:
import pickle
import numpy as np
from tensorflow.data import Dataset
from tensorflow.data.experimental import save as save_tf_data_set

In [3]:
def read_np_data(f_name):
    with open(f_name, 'rb') as f:
        np_data = np.load(f, allow_pickle=True)
    return np_data

In [48]:
batch_size=128

In [49]:
input_data = read_np_data('input_ids_np.npy')
attention_mask = read_np_data('attention_mask_np.npy')
labels = read_np_data('labels.npy')

In [50]:
input_data.shape, attention_mask.shape, labels.shape

((8529, 512), (8529, 512), (8529, 5))

In [51]:
dataset = Dataset.from_tensor_slices((input_data, attention_mask, labels))

In [52]:
def map_data(in_data, atten_mask, labels):
    return {'input_data': in_data, 'attention_mask': atten_mask}, labels

In [53]:
dataset_mapped = dataset.map(map_data)

In [54]:
dataset_mapped

<MapDataset shapes: ({input_data: (512,), attention_mask: (512,)}, (5,)), types: ({input_data: tf.int32, attention_mask: tf.int32}, tf.float64)>

In [55]:
dataset_mapped.take(1)

<TakeDataset shapes: ({input_data: (512,), attention_mask: (512,)}, (5,)), types: ({input_data: tf.int32, attention_mask: tf.int32}, tf.float64)>

In [56]:
dataset_mapped = dataset_mapped.shuffle(buffer_size=len(dataset))

In [57]:
dataset_mapped = dataset_mapped.batch(batch_size=batch_size, drop_remainder=True)

In [58]:
train_size = int(len(input_data)/batch_size * 0.9)
train_size

59

In [59]:
train_dataset = dataset_mapped.take(train_size)
valid_dataset = dataset_mapped.skip(train_size)

In [60]:
save_tf_data_set(train_dataset, 'train_dataset')
save_tf_data_set(valid_dataset, 'valid_dataset')

In [61]:
element_spec = train_dataset.element_spec

In [62]:
with open('element_spec.pkl', 'wb') as f:
    pickle.dump(element_spec, f)